In [ ]:
%%cmd
c:\\adb\\adb.exe shell service check isms

In [ ]:
%%cmd
c:\\adb\\adb.exe shell service call isms option args

In [ ]:
%%cmd 
 shell service call isms 7 i32 0 s16 "com.android.mms.service" s16 "+12397705265" s16 "null" s16 "'THIS IS A TEST'" s16 "null" s16 "null"


In [ ]:
def getSummary(json):
    out = {}
    for area in json:
        if 'Cases Reported' in area:
            out[area['Jurisdiction']] = area['Cases Reported'] if area['Cases Reported'] != 'None' else 0
    return out

In [ ]:
def getTop(json,c=3):
    sorted_d = sorted(json.items(), key=operator.itemgetter(1),reverse=True)
    return dict(sorted_d[:c])

In [ ]:
def getMessage(json, old):
    data = getChanges(json,old)
    out = []
    for m in data:

        out.append('{}: new:{} | old:{}'.format(m,data[m],old[m] if m in old else 'None'))
    return '\n'.join(out)

In [ ]:

def splitText(text_message):
    for start in range(0,len(text_message),100):
        end = start+100
        print(start,end)
        if end > len(text_message):
            end = len(text_message)
        yield(text_message[start:end])

In [ ]:
def getChanges(json, old):
    changes = {}
    print(json)
    print(old)
    for item in json:
        print(item)
        if item not in old or json[item] != old[item]:
            changes[item]=json[item]
    return changes

In [ ]:
import subprocess, os
def adbCommand(command):
    cmd = ["c:\\adb\\adb.exe"] + command
    print(cmd)
    p = subprocess.Popen(
        cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE
    )
    out, err = p.communicate()
    return out,err


def sendText(phone_number, text_message):
    out,err =  adbCommand(["shell","service","call","isms","7", "i32", "0", "s16", '"com.android.mms.service"', 's16', "+{}".format(phone_number), 's16', '"null"', 's16', "'{}'".format(text_message), 's16', "null", 's16', "null"])
    print('err', err)
    print('out', out)

In [ ]:

import time
from datetime import datetime
import requests
import operator

In [ ]:
old = {}

In [ ]:
url = "https://www.cdc.gov/coronavirus/2019-ncov/map-cases-us.json"
while True:
    now = datetime.now().strftime('%m_%d_%Y-%H_%M_%S')
    print(now,end='')
    res = requests.get(url)
    json = res.json()['data']
    json = getSummary(json)
    if json != old:
        for msg in splitText("DATA CHANGED:"+getMessage(json,old)):
            print('sending:',msg)
            sendText('12397705265',msg)
        with open('./data/{}.json'.format(now),'wb') as f:
            f.write(res.content)
        old = json
        print('found changes')
        print(json)
        print()
    else:
        print('no changes')
    
    time.sleep(60)

In [ ]:
sendText('12397705265',"data changed\n{}".format(getMessage(json,old)))

In [ ]:
json

In [ ]:
old['New York'] = 9

In [ ]:
testText('sasdf','test'*160)